In [1]:
!pip install -qU git+https://github.com/lenguajenatural-ai/autotransformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.

In [2]:
from autotransformers import AutoTrainer, DatasetConfig, ModelConfig, ResultsPlotter
from transformers import EarlyStoppingCallback
from transformers import Seq2SeqTrainer, PegasusForConditionalGeneration, ProphetNetForConditionalGeneration, XLMProphetNetForConditionalGeneration, MT5ForConditionalGeneration

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
fixed_train_args = {
    "evaluation_strategy": "epoch",
    "num_train_epochs": 5,
    "do_train": True,
    "do_eval": True,
    "logging_strategy": "epoch",
    "save_strategy": "epoch",
    "save_total_limit": 2,
    "seed": 69,
    "bf16": False,
    "dataloader_num_workers": 16,
    "load_best_model_at_end": True,
    "optim": "adafactor",
    #"max_steps": 1 # NOTE: This is added for the purpose of the tutorial.
}

In [4]:
sum_config = {
    "seed": 44,
    "direction_optimize": "maximize",
    "metric_optimize": "eval_rouge2",
    "callbacks": [EarlyStoppingCallback(1, 0.00001)],
    "fixed_training_args": fixed_train_args
}

sum_config.update(
    {
        "dataset_name": "AlexanderBenady/generated_lectures",
        "alias": "AlexanderBenady/generated_lectures",
        "retrain_at_end": False,
        "task": "seq2seq",
        "hf_load_kwargs": {"path": "AlexanderBenady/generated_lectures", "name": "default"},
        'text_field' : 'Lecture',
        "label_col": "Summary",
        'summary_field' : "Summary",
        "num_proc": 16,
        "split" : True}
)

sum_config = DatasetConfig(**sum_config)

In [5]:
sum_config

DatasetConfig(dataset_name='AlexanderBenady/generated_lectures', alias='AlexanderBenady/generated_lectures', task='seq2seq', fixed_training_args={'evaluation_strategy': 'epoch', 'num_train_epochs': 5, 'do_train': True, 'do_eval': True, 'logging_strategy': 'epoch', 'save_strategy': 'epoch', 'save_total_limit': 2, 'seed': 69, 'bf16': False, 'dataloader_num_workers': 16, 'load_best_model_at_end': True, 'optim': 'adafactor'}, is_multilabel=False, multilabel_label_names=[], hf_load_kwargs={'path': 'AlexanderBenady/generated_lectures', 'name': 'default'}, type_load='json', files=None, data_field='data', partial_split=False, split=True, label_col='Summary', val_size=0.15, test_size=0.15, pre_func=None, remove_fields_pre_func=False, squad_v2=False, text_field='Lecture', is_2sents=False, sentence1_field=None, sentence2_field=None, summary_field='Summary', callbacks=[<transformers.trainer_callback.EarlyStoppingCallback object at 0x7b20d335fa30>], metric_optimize='eval_rouge2', direction_optimize

In [6]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_categorical(
            "learning_rate", [3e-5, 5e-5, 7e-5, 2e-4]
        ),
        "num_train_epochs": trial.suggest_categorical(
            "num_train_epochs", [6]
        ),
        "per_device_train_batch_size": trial.suggest_categorical(
            "per_device_train_batch_size", [3]),
        "per_device_eval_batch_size": trial.suggest_categorical(
            "per_device_eval_batch_size", [3]),
        "gradient_accumulation_steps": trial.suggest_categorical(
            "gradient_accumulation_steps", [8]),
        "warmup_ratio": trial.suggest_categorical(
            "warmup_ratio", [0.08]
        ),
    }

In [7]:
def preprocess_function(examples, tokenizer, dataset_config):
    model_inputs = tokenizer(
        examples[dataset_config.text_field],
        truncation=True,
        max_length=128
    )
    labels = tokenizer(
        text_target=examples[dataset_config.summary_field],
        max_length=dataset_config.max_length_summary,
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
pegasus_config = ModelConfig(
    name="google/pegasus-cnn_dailymail",
    save_name="pegasus",
    hp_space=hp_space,
    num_beams=4,
    trainer_cls_summarization=Seq2SeqTrainer,
    model_cls_summarization=PegasusForConditionalGeneration,
    custom_tokenization_func=preprocess_function,
    n_trials=8,
    random_init_trials=1
)

prophet_config = ModelConfig(
    name="microsoft/prophetnet-large-uncased-squad-qg",
    save_name="prophet",
    hp_space=hp_space,
    num_beams=4,
    trainer_cls_summarization=Seq2SeqTrainer,
    model_cls_summarization=ProphetNetForConditionalGeneration,
    custom_tokenization_func=preprocess_function,
    n_trials=8,
    random_init_trials=1
)

xlm_prophet_config = ModelConfig(
    name="microsoft/xprophetnet-large-wiki100-cased",
    save_name="xlm-prophet",
    hp_space=hp_space,
    num_beams=4,
    trainer_cls_summarization=Seq2SeqTrainer,
    model_cls_summarization=XLMProphetNetForConditionalGeneration,
    custom_tokenization_func=preprocess_function,
    n_trials=8,
    random_init_trials=1
)

mt5_config = ModelConfig(
    name="google/mt5-large",
    save_name="mt5",
    hp_space=hp_space,
    num_beams=4,
    trainer_cls_summarization=Seq2SeqTrainer,
    model_cls_summarization=MT5ForConditionalGeneration,
    custom_tokenization_func=preprocess_function,
    n_trials=8,
    random_init_trials=1
)


In [9]:
autotrainer = AutoTrainer(
    model_configs=[pegasus_config, prophet_config, xlm_prophet_config, mt5_config],
    dataset_configs=[sum_config],
    metrics_dir="mlsum_multilingual_models",
    metrics_cleaner="metrics_mlsum"
)

In [ ]:
results = autotrainer()
print(results)

Iterating over datasets...:   0%|          | 0/1 [00:00<?, ?it/s]
Trying models on dataset AlexanderBenady/generated_lectures:   0%|          | 0/1 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

prophetnet.tokenizer:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,4.777400,3.408466,22.579400,11.015000,19.444000,21.735900,20.000000
1,3.277500,3.050499,21.887700,12.340200,19.484900,20.878700,20.000000
2,2.859500,2.963872,21.293200,13.067600,19.407900,20.216000,20.000000
3,2.615200,2.894737,22.274500,13.143500,19.868800,20.940000,20.000000
4,2.443600,2.881255,22.147800,13.602400,20.170200,21.284600,20.000000
5,2.286900,2.880265,22.192200,13.317000,19.953500,21.080800,20.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,4.767900,3.419069,22.168800,12.031100,18.921700,21.022400,20.000000
1,3.262400,3.096627,21.461500,12.502500,19.367700,20.331200,20.000000
2,2.851600,2.960370,21.433200,12.780100,19.379900,20.187800,20.000000
3,2.605900,2.887352,22.327900,13.263100,19.916500,21.050600,20.000000
4,2.436300,2.877595,22.412700,13.706600,19.961700,21.270400,20.000000
5,2.280000,2.875173,22.026100,13.318800,19.735900,20.888400,20.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local

Epoch,Training Loss,Validation Loss
